In [13]:
# necessary imports
# !pip install pm4py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import pm4py


np.random.seed(123)

## Import data

Specify the names of bpmn diagram (should be stored in bpmn_diagrams), log file (as a csv file, should be stored in logs_folder) and bpmn specifics (should be stored in bpmn_specifics folder as csv files)

In [70]:
# EDIT BELOW #
##############
bpmn_diagram = 'diagram6.bpmn'
bpmn_specifics = 'kolokwium.csv'
logs = 'TODO'
############
# END EDIT #

path_to_bpmn_diagram = 'bpmn_diagrams/' + bpmn_diagram
path_to_specifics = 'bpmn_specifics/' + bpmn_specifics

# loading bpmn into python through pm4py
bpmn = pm4py.read_bpmn(path_to_bpmn_diagram)
# transfomring it into petri net
net, im, fm = pm4py.convert_to_petri_net(bpmn)
# simulating logs with pm4py
log = pm4py.play_out(net, im, fm)


In [71]:
from pm4py.objects.conversion.log.variants import to_data_frame
log_df = to_data_frame.apply(log)

In [72]:
log_df.shape

(2516, 3)

In [73]:
log_df.head(10)

,concept:name,time:timestamp,case:concept:name
0,Display Balance,1970-04-26 18:46:40+00:00,0
1,Display Balance,1970-04-26 18:46:41+00:00,1
2,Insert Card,1970-04-26 18:46:42+00:00,2
3,Select Interaction,1970-04-26 18:46:43+00:00,2
4,Check For Further Interactions,1970-04-26 18:46:44+00:00,2
5,Insert Card,1970-04-26 18:46:45+00:00,3
6,Select Interaction,1970-04-26 18:46:46+00:00,3
7,Select Amount,1970-04-26 18:46:47+00:00,3
8,Check For Further Interactions,1970-04-26 18:46:48+00:00,3
9,Display Balance,1970-04-26 18:46:49+00:00,4


In [74]:
log_df['concept:name'].unique()

# start --> t1 ---> t2 ---> t3/t4 ---> t5 -/ dummy_task /-> stop

array(['Display Balance', 'Insert Card', 'Select Interaction',
       'Check For Further Interactions', 'Select Amount',
       'Charge Account', 'Prepare Cash', 'Issue Money'], dtype=object)